In [ ]:
#! mkdir ~/.kaggle
#! cp kaggle.json ~/.kaggle/
#! chmod 600 ~/.kaggle/kaggle.json
#! kaggle datasets list
#!pip install --upgrade --force-reinstall --no-deps kaggle
#!kaggle competitions download -c santander-value-prediction-challenge
#! mkdir santander
#! unzip santander-value-prediction-challenge.zip -d santander

# **BUSINESS CONTEXT OF THE PROBLEM**

INTRODUCTION:Santander value prediction challenge is a third competition in a row wherein santander has taken a step further and curated a task to determine the value of transaction with a potential customers and  by having the idea of rough potential transaction  institutions may speculate/predict the type of services a person might need and in turn they can personalise their services for each customer . An amount is a continuous target variable here and problem is a "**SUPERVISEDD LEARNING REGRESSION TASK**"


#MAPPING BUSINESS PROBLEM TO A MACHINE LEARNING PROBLEM
FOR A GIVEN DATASET X [n_rows,n_features] predict continuous target variable Y.
such that (Y-Y_hat) is minimised 

> 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
! mkdir santander

In [ ]:
train=pd.read_csv('../input/santander-value-prediction-challenge/train.csv')
print("train",train.shape)
train.head(2)

In [ ]:
#lets load the test data
#test=pd.read_csv('/content/santander/test.csv')
#print("test",test.shape)

#sample_submission=pd.read_csv('/content/santander/sample_submission.csv')
#print("sample_submission",sample_submission.shape)

# EXPLORATORY DATA ANALYSIS

In [ ]:
train.head(2)
#By looking at the dataset overview it clearly shows that data has sparcity and most of the feature we see there is a huge gap between two values belonging to same features 
#Example  22000000 and 0.0 , 14800000 and 0.0 

In [ ]:
train.info()
#As we can see train data has most of features are float and int and one object feature

In [ ]:
#lets print the duplicate columns 
print(len(train.columns))    # total number of columns
print(len(set(train.columns)))      # unique number of columns 
# since number of unique features = total columns , hence there is not duplicate column


In [ ]:
#Now lets explore the number of constant feautres as due to constant values they are not capturing any varience
filt=(train.nunique()==1) # feature wise unique value count
train.nunique()[filt]     # number features have single values
print("constant features:",train.nunique()[filt].shape[0])

In [ ]:
# Lests drop the constant features
col=list(train.nunique()[filt].reset_index()['index'])
train=train.drop(columns=col)
print('train',train.shape)


In [ ]:
df=train.T.drop_duplicates()
# train.T had (4737 rows × 4459 columns) and after removal of duplicates it has reduced to 4733 rows × 4459 columns 

In [ ]:
df.shape
df.columns
train=df.T
print("train after duplicate columns removal",train.shape)

In [ ]:
#lets print the duplicate columns 
print("total number of columns",len(train.columns))    # total number of columns
print("unique columns",len(set(train.columns)))      # unique number of columns 
# since number of unique features = total columns , hence there is not duplicate column


In [ ]:
#Duplicate features
dup={}
for i,col in enumerate(train.columns):
  to_remove=[]
  for j in range(i+1,len(train.columns)):
    if np.all(train.iloc[:,i].values == train.iloc[:,j].values)==True:
      to_remove.append(train.columns[j])
  if len(to_remove)!=0:
    dup[col]=to_remove
  else:
   continue   

In [ ]:
#Duplicates features = ['d60ddde1b','acc5b709d', 'f333a5f60','f8d75792f','912836770','f333a5f60']
#dup
#drop duplicate features
#train=train.drop(columns=['d60ddde1b','acc5b709d', 'f333a5f60','f8d75792f','912836770','f333a5f60'])

In [ ]:
#lets create target variable 
y_train=train['target']
print('y_train',y_train.shape)
train.columns[2:]

In [ ]:
#Check for null values
null_features=[]
for f in train.columns:
  if np.any(train[f].isnull().values) == True:
    null_features.append(f)
  else:
    continue
print("number of null values in train :",null_features) 
print(train.shape)
#print(test.shape)

In [ ]:
#As a part of EDA lets quantify the sparcity of train data 
flat_train=train.iloc[:,1:].to_numpy().flatten() # flatten the train data
print("Total values:",flat_train.shape[0])  ## tota non zero values in dataset

# Calculate total number of non zero values
sparcity = flat_train[flat_train!=0].shape[0]/flat_train.shape[0] * 100
print("Perentage of non_zero values:",sparcity,"%")

# so from sparcity values we can see that aprox 3.3% values are non zero

#Lets depict it on plot

import plotly.express as px
  
# Random Data
random_x = [flat_train.shape[0],flat_train[flat_train!=0].shape[0]]
names = ['Total', 'non_zero']
fig = px.pie(values=random_x, names=names)
fig.show()
# ONLY 3.23% VALUES ARE NON ZERO IN TRAIN DATA REST ARE ZEROS

# EDA(b): FEATURE SELECTION

In [ ]:
from sklearn.preprocessing import minmax_scale
train_scaled = minmax_scale(train.iloc[:,2:], axis = 0)
train_scaled.shape

In [ ]:
# Train test split ,since we could also use validation_ratio=0.9 but using that aproach in may lead to feeding of test data
# while training so its better to split it before and define as a validation_data
from sklearn.model_selection import train_test_split
x_train, x_test, y_tr, y_ts = train_test_split(train_scaled, y_train, train_size = 0.7, random_state = 42)
print("x_train",x_train.shape)
print("x_test",x_test.shape)
print("y_tr",y_tr.shape)
print("y_ts",y_ts.shape)


In [ ]:
#Now lets understand the distribution of target variable
plt.figure(figsize=(8,8))
sns.histplot(data=y_tr)
plt.grid()
plt.show()
#since data is right skewed and we know from statistical point of view that data should be normal or close to normal
#that can be done by box cox transform or in simple term by log normalising the data

In [ ]:
import seaborn as sns
x=np.arange(len(y_train))
plt.figure(figsize=(10,10))
sns.histplot(np.log1p(y_tr.astype(float)),kde=True)
plt.show()
#Now we can see that we have normalised data and it is more close to normal distribution
# which could be an advantage for better generalisation

In [ ]:
#BEFORE DOING FEATURE SELECTION WE NEED TO GET FEATURE IMPORTANCES 
#LETS TRAIN A RANDOM FOREST MODEL TO GET THOSE
import pickle
import os
#path="/content/drive/MyDrive/APPLIED AI ASSIGNMENTS ipynb/CASE STUDY 1/baseline_model"
path="./santander/baseline_model_after70% split"

if not os.path.isfile(path):
  from sklearn.ensemble import RandomForestRegressor
  regressor=RandomForestRegressor()
  regressor.fit(x_train,y_tr)
  pickle.dump(regressor, open(path, 'wb'))
else:
  regressor=pickle.load(open(path, 'rb'))


from sklearn.metrics import mean_squared_log_error
pred_test=regressor.predict(x_test)
rmsle_test=np.sqrt(mean_squared_log_error(y_ts,pred_test))  

pred_train=regressor.predict(x_train)
rmsle_train = np.sqrt(mean_squared_log_error(y_tr,pred_train))
print("rmsle_test",rmsle_test)
print("rmsle_train",rmsle_train)

from sklearn.metrics import r2_score
print("R_square_train",r2_score(y_tr, pred_train))
print("R_squared_test",r2_score(y_ts,pred_test))

#Plots
residual_train = (y_tr - pred_train)
residual_test = (y_ts - pred_test)

f, (ax1, ax2) = plt.subplots(1, 2,figsize=(15,7))
ax1.scatter(pred_train, residual_train)
ax1.set_title('Target vs Residual plot')
ax2.scatter(pred_test,residual_test )
ax2.set_title('Target vs Residual plot')
############################################

print("\nResults after log transformation of target variable")
#path1="/content/drive/MyDrive/APPLIED AI ASSIGNMENTS ipynb/CASE STUDY 1/baseline_model_second_attempt"
path1="./santander/baseline_model_second_attempt_after70%split"

if not os.path.isfile(path1):
  from sklearn.ensemble import RandomForestRegressor
  regressor2=RandomForestRegressor()
  regressor2.fit(x_train,np.log1p(y_tr.astype(float)))
  pickle.dump(regressor, open(path1, 'wb'))
else:
  regressor2=pickle.load(open(path1, 'rb'))


from sklearn.metrics import mean_squared_log_error
pred_test=regressor2.predict(x_test)
rmsle_test=np.sqrt(mean_squared_log_error(y_ts,np.exp(pred_test)))  

pred_train=regressor2.predict(x_train)
rmsle_train = np.sqrt(mean_squared_log_error(y_tr,np.exp(pred_train)))
print("\nrmsle_test",rmsle_test)
print("rmsle_train",rmsle_train)


residual_train = (y_tr - np.exp(pred_train))
residual_test = (y_ts - np.exp(pred_test))

f, (ax3, ax4) = plt.subplots(1, 2,figsize=(15,7))
ax3.scatter(pred_train, residual_train)
ax3.set_title('Target vs Residual plot')
ax4.scatter(pred_test,residual_test )
ax4.set_title('Target vs Residual plot')

from sklearn.metrics import r2_score
print("R_square_train",r2_score(y_tr, np.exp(pred_train) ))
print("R_squared_test",r2_score(y_ts,np.exp(pred_test)))

#Rmsle value for train and test data are far apart i.e model is overfitting but severity of overfit increased on training log transformed data'
#although test as well as train error has reduces significantly but Rmsle values are also far apart.

In [ ]:
importance=regressor.feature_importances_
print("feature_importances_",regressor.feature_importances_)
print("featues:",regressor.n_features_)
args=importance.argsort()[::-1][:20]
top_20=importance[args]
plt.style.use('dark_background')
plt.figure(figsize=(40,20))
#plt.grid()
plt.barh(y=np.arange(0,20), width=top_20,color='r')
plt.xlabel("Importances")
plt.ylabel("Features")
plt.tight_layout
plt.title("Top 20 Feature importances")
for i in range(0,20):
  plt.annotate("{}".format(np.round(top_20[i],4)),xy=(top_20[i]+0.001,i),size=20)
plt.show()
# top 20 important features along with their importance coefficient
# for further we will do univeriate as well as bivariate analysis 

In [ ]:
args
data=train.iloc[:,args]
data.columns

In [ ]:
#Correlation heatmap for imporatant features
data=train.iloc[:,args].astype(float)
data.corr()
plt.figure(figsize=(25,15))
sns.heatmap(data.corr(method='spearman'),vmin=0,vmax=1,annot=True)
plt.show()

#As we can see there is some correlation between important features but they all are not much significant with respect to
# statistical standpoint

In [ ]:
#data.columns
 
#top_20 = ['d401c2b4a', 'e7c0a50e8', 'f6240919f', '268040457', 'dbc48d37c',
#       '22fbf6997', 'b517115d3', 'd08d1fbe3', '46e10e042', '01005e5de',
#       '0933930b4', '04b88be38', '413bbe772', '9c502dcd9', 'f52a82e7f',
#       'e176a204a', 'd79736965', '23c780950', '1bf8c2597', 'ba480f343'] 

top_20 = data.columns
top_20

In [ ]:
#Data is highly effected by outliers as we can see all the 15 features are facing the issue of outlier on the positive side 
plt.style.use('dark_background')
plt.figure(figsize=(20,10))
#data.boxplot(figsize=(15,20),color='r')
sns.boxplot(data=data,color='b',showfliers=True,orient='vertical').set_ylim(-10e4,10e6)
plt.xlabel("Values")
plt.ylabel("Features")
plt.tight_layout()
plt.show()  
# From boxplot we can see that there are lot of outliers in all the top important features.again for all practical using rmse as a loss 
# function is a good choice for this type of data and since features have very diffent scales ,it need feature scaling

In [ ]:
plt.figure(figsize=(30,12))
sns.violinplot(data=data,orient='vertical').set_ylim(-10e6,10e7)
plt.tight_layout()
plt.ylabel("Values")
plt.xlabel("Features")
plt.title("Violin Plot")
plt.tight_layout()
plt.show()
#Distributions are highly skewed about their means and are symmetric. As we saw in box plot data has outliers same thing is observed here also.

In [ ]:
y_train
#y_tr
#train.shape
#args
#data=train.iloc[:,args]
data.shape,data.columns
#cors=np.array(data.corrwith(np.log1p(y_train.astype(float))))cors
#cors=np.array(data.corrwith(np.log1p(y_train.astype(float))))
#cors

In [ ]:
#Apart from correlation plot lets print the correlation of these importance features with target variable 

args
data=train.iloc[:,args].astype(float)
data.columns

cors=np.array(data.corrwith(np.log1p(y_train.astype(float))))
#cors=np.array(data.corrwith(y_train))
cols=data.columns
print("Correlation coeff with target variable:\n",dict(zip(cols,cors)))
cols=data.columns
plt.figure(figsize=(40,15))
plt.bar(x=cols,height=cors,width=0.5,color='r')
for i in range(20):
  plt.annotate("{}".format(np.round(cors[i],4)),xy=(i,np.round(cors[i],4)),size = 20)
plt.xlabel("Features")
plt.ylabel("Correlation coeff")
plt.title("Featues vs Correlation plot")
plt.show()  

# Out of 15 ,three shows high positive correlation with the target variable []


In [ ]:
data.columns
plt.figure(figsize=(17,12))
for i,col in enumerate(data.columns):
  plt.subplot(5,4,i+1)
  sns.scatterplot(x=np.arange(len(data[col])),y=data[col])
  plt.tight_layout()
plt.show()
#most of features are less than 1*1e6 and outlier are there but threse number is less in most of the features and rest of the 
#features there is some uniformity .from this observation we can conclude that seriously need feature scaling 

In [ ]:
plt.figure(figsize=(20,8))
sns.pairplot(data.iloc[:,:10])
plt.show()
#

In [ ]:
# Import libraries for autoencoders 
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import MinMaxScaler, StandardScaler

from keras.models import Sequential, Model
from keras.layers import Input, Dense, Dropout, BatchNormalization

from sklearn.model_selection import train_test_split
from sklearn import metrics

from imblearn.over_sampling import RandomOverSampler

import os
import gc 
#print(os.listdir("../input"))

In [ ]:
print("train",train.shape)
train.head(2)

In [ ]:

print("train_scaled",train_scaled.shape)

In [ ]:
x_train.shape

In [ ]:
#Lests use Autoencoders to reduce Dimensions and select important features
path_train = "./santander/engineered_features_train_third_attempt"
path_test  = "./santander/engineered_features_test_third_attempt"
#path = "/content/drive/MyDrive/APPLIED AI ASSIGNMENTS ipynb/CASE STUDY 1/Encoder_second_attempt"

if not os.path.isfile(path_test):
  n_epochs=30
  input_data = Input(shape=(4730,))
  encoded = Dense(2048, activation='relu',name='encoded_layer1')(input_data)
  encoded= Dense(1024, activation='relu',name='encoded_layer2')(encoded)
  #encoded = BatchNormalization()(encoded)

  encoded= Dense(512, activation='relu',name='encoded_layer3')(encoded)
  encoded= Dense(256, activation='relu',name='encoded_layer4')(encoded)
  #encoded = BatchNormalization()(encoded)

  encoded= Dense(128, activation='relu',name='encoded_layer5')(encoded)
  encoded = BatchNormalization()(encoded)


  decoded = Dense(128, activation='relu',name="decoder1")(encoded)
  decoded = Dense(256, activation='relu',name='decoder2')(decoded)
  #decoded = BatchNormalization()(decoded)

  decoded = Dense(512, activation='relu')(encoded)
  #decoded = BatchNormalization()(decoded)

  decoded = Dense(1024, activation='relu')(encoded)
  #decoded = BatchNormalization()(decoded)

  decoded = Dense(2048, activation='relu')(encoded)
  #decoded = BatchNormalization()(decoded)

  decoded = Dense(4730, activation='relu')(decoded)
  #decoded = BatchNormalization()(decoded)

  autoencoder = Model(input_data, decoded)
  autoencoder.compile(optimizer='adagrad', loss=["mean_squared_error"]) #,"mean_squared_error"]) #loss='mean_squared_error')

# Encoding Train Data
  autoencoder.fit(x_train, x_train, epochs = n_epochs, batch_size = 45, shuffle = True, validation_data= (x_test,x_test)) #,validation_split=0.20)
  #autoencoder.fit(np.array(train.iloc[:,2:]),np.array(train.iloc[:,2:]), epochs = 15, batch_size = 30, shuffle = False, validation_split=0.2) #,validation_split=0.20)
  encoder = Model(inputs = input_data, outputs = encoded)

  #pickle.dump(encoder, open(path, 'wb'))
  encoded_train = pd.DataFrame(encoder.predict(x_train))
  encoded_train = encoded_train.add_prefix('feature_')

  encoded_test = pd.DataFrame(encoder.predict(x_test))
  encoded_test = encoded_test.add_prefix('feature_')

  pickle.dump(encoded_train, open(path_train, 'wb'))
  pickle.dump(encoded_test, open(path_test, 'wb'))
else:
  encoded_train = pickle.load(open(path_train,'rb'))
  encoded_test = pickle.load(open(path_test,'rb'))

#print("encoded_test",encoded_test.shape)
print("encoded_train",encoded_train.shape)  

In [ ]:
#CONCLUSION

#since we have done detailed analysis of high dimensional data
#1.data is highly sparse as it has only 3.23% non zero rest close to 97% data is zero.
#2.data feature has lot of scale imbalace which we fixed later by feature scaling
#3.of the 4993 features 256 were constant feature and 6 duplicate features i.e from PCA(pricipal component analysis) perspective they will capture zero varience hence its better 
#to impute them
#4.data set has no NA values
#On applying dimensionality reduction using encoder part of autoencoder sparcity of data is reduced and dimensions are also came down to 129 still after using autoencoders
# interpretability is a issue as in PCA we were able to capture 99.6% of variance in 2000 features out of aprox 4700 features but here reconstruction error is the only measure we 
# can look at and in our case reconstrucion error was coming consistenly constant during the last phase of training which shows consistent improvement.
#6.y_train the target variable is right skewed so to make it close to normal we did log transformation.


# MODELLING AND 'HYPERPARAMETER TUNING'


In [ ]:

top_20

In [ ]:
train[top_20]

In [ ]:
#Concatenating top 20 and encoded_test
cols_test=pd.DataFrame(x_test,columns=train.iloc[:,2:].columns).loc[:,top_20]
Data_imp_test = pd.concat([encoded_test,cols_test],axis=1)
test_columns=Data_imp_test.columns
print("test",Data_imp_test.shape)

cols_train=pd.DataFrame(x_train,columns=train.iloc[:,2:].columns).loc[:,top_20]
Data_imp_train=pd.concat([encoded_train,cols_train],axis=1)
train_columns= Data_imp_train.columns
print("train",Data_imp_train.shape)
Data_imp_train.head(2)


#from sklearn.preprocessing import minmax_scale
Data_imp_train = minmax_scale(Data_imp_train, axis = 0)
Data_imp_test = minmax_scale(Data_imp_test, axis = 0)


Data_imp_train = pd.DataFrame(Data_imp_train,columns=train_columns)
Data_imp_test = pd.DataFrame(Data_imp_test,columns=test_columns)
print("Data_imp_train",Data_imp_train.shape)
print("Data_imp_test",Data_imp_test.shape)

#print(Data_imp_train.head())
#print(Data_imp_test.head())

In [ ]:
#Import and train lightGBM model

#import lightgbm as lgb
#from sklearn.metrics import mean_squared_log_error
#gbm = lgb.LGBMRegressor()

#model=gbm.fit(Data_imp_train,y_tr.astype(float))
#model.best_iteration
#y_pred_train=model.predict(Data_imp_train)
#print(y_pred_train)
#rmsle_train = np.sqrt(mean_squared_log_error(y_tr,np.clip(y_pred_train,a_min=0,a_max=None)))
#print("Rmsle on train data",rmsle_train)

#y_pred_test = model.predict(Data_imp_test)
#print("\ny_pred_test",y_pred_test)
#rmsle_test=np.sqrt(mean_squared_log_error(y_ts,np.clip(y_pred_test,a_min=0,a_max=None)))
#print("\ny_test",sample_submission.target.values)
#print("\nRmsle on test data",rmsle_test)

#from sklearn.metrics import r2_score
#print("\nR_square_train",r2_score(y_tr, y_pred_train))
#print("\nR_squared_test",r2_score(y_ts,y_pred_test))
#print("###"*10)


#residual_train = (y_tr - y_pred_train)
#residual_test = (y_ts - y_pred_test)

#f, (ax3, ax4) = plt.subplots(1, 2,figsize=(15,7))
#ax3.scatter(pred_train, residual_train)
#ax3.set_title('Sharing Y axis')
#ax4.scatter(pred_test,residual_test )
#ax4.set_title('Sharing Y axis')

###################################################################
###################################################################

import lightgbm as lgb
from sklearn.metrics import mean_squared_log_error
gbm = lgb.LGBMRegressor()

model=gbm.fit(Data_imp_train,np.log1p(y_tr.astype(float)))
y_pred_train=model.predict(Data_imp_train)
print(np.expm1(y_pred_train))
rmsle_train = np.sqrt(mean_squared_log_error(y_tr,np.expm1(y_pred_train)))
print("Rmsle on train data",rmsle_train)

y_pred_test = model.predict(Data_imp_test)
print("\ny_pred_test",np.expm1(y_pred_test))
rmsle_test=np.sqrt(mean_squared_log_error(y_ts,np.expm1(y_pred_test)))
print("\nRmsle on test data",rmsle_test)

from sklearn.metrics import r2_score
print("\nR_square_train",r2_score(y_tr, np.expm1(y_pred_train) ))
print("\nR_squared_test",r2_score(y_ts,np.expm1(y_pred_test)))

residual_train = (y_tr - np.exp(y_pred_train))
residual_test = (y_ts - np.exp(y_pred_test))

f, (ax3, ax4) = plt.subplots(1, 2,figsize=(15,7))
ax3.scatter(pred_train, residual_train)
ax3.set_title('train vs residual plot')
ax4.scatter(pred_test,residual_test )
ax4.set_title('Test vs residual plot')

#Baseline Rmsle score train : 0.07089089515547918 and test : 0.07135105539024963

In [ ]:
from sklearn.model_selection import KFold
def light_gbm_model_run(train_x, train_y, validation_x, validation_y, test_x,**kwargs):
    params = {
        "objective" : "regression",
        "metric" : "rmse",
        'n_estimator':67,
        "num_leaves" : 45,
       "learning_rate" :0.01,
        "max_depth":5,
       # "bagging_fraction" : 0.5,
       # "feature_fraction" : 0.5,
       # "bagging_frequency" : 6,
        "reg_alpha":5,
        "reg_lambda":64,
        #"bagging_seed" : 42,
        "verbosity" : -1,
        "seed": 42
    }
    # Given its a regression case, I am using the RMSE as the metric.

    gbm_train = lgb.Dataset(Data_imp_train, label=np.log1p(y_tr.astype(float)))
    gbm_val = lgb.Dataset(Data_imp_test, label=np.log1p(y_ts.astype(float)))
    val_result_lgbm = {}

    model_light_gbm = lgb.train(params,gbm_train,5000,
                      valid_sets=[gbm_train, gbm_val],**kwargs,
                      early_stopping_rounds=100,
                      verbose_eval=20,
                      evals_result=val_result_lgbm)

    pred_test_light_gbm = np.expm1(model_light_gbm.predict(test_x, num_iteration=model_light_gbm.best_iteration ))

    return pred_test_light_gbm, model_light_gbm, val_result_lgbm,model_light_gbm.best_iteration

n_splits=3
kf = KFold(n_splits=3, shuffle=True, random_state=45)
pred_test_full = 0
for dev_index, val_index in kf.split(Data_imp_train):
    dev_X, val_X = Data_imp_train.loc[dev_index,:], Data_imp_train.loc[val_index,:]
    dev_y, val_y = np.log(y_train[dev_index].astype(float)), np.log(y_train[val_index].astype(float))
    pred_test, model, evals_result,best_iter = light_gbm_model_run(dev_X, dev_y.astype(float), val_X, val_y.astype(float), Data_imp_test)
    print("pred_test",len(pred_test))
    print("\nbest parameters after training",best_iter)
    pred_test_full += pred_test
pred_test_full /= n_splits
pred_test_full = np.exp(pred_test_full,dtype=np.float128)

from sklearn.metrics import r2_score
print("\nR_square_train",r2_score(y_tr, np.exp(y_pred_train.astype(float))))
#print("\nR_squared_test",r2_score(np.log1p(sample_submission.target.values),np.expm1(y_pred_test)))

plt.figure(figsize=(8,8))
plt.grid()
plt.plot(evals_result['training']['rmse'],label='training')
plt.plot(evals_result['valid_1']['rmse'],label='test')
plt.xlabel("iterations")
plt.ylabel("error")
plt.legend()
plt.show()

#validation errors are reducing initially but after crossing 65 - 70 estimators it started overfitting 
# validation  loss has range (1.74,1.59)

# **XGboost model training**